<a href="https://colab.research.google.com/github/Avni05-glitch/Avni05-codsoft/blob/main/RAG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pinecone-client --upgrade
import pinecone
import openai
from pinecone import Pinecone, PodSpec  # Import PodSpec

# Initialize Pinecone using Pinecone class
# **Verify the environment name**  Make sure it's accurate according to Pinecone's documentation.
# Try using 'us-east4-gcp' or 'gcp-starter' instead of 'us-east1-gc'
pinecone_client = Pinecone(api_key="pcsk_5n25vn_9mtw4PWydANfPXtrWz6uVAa6CQM6zJrHb61DnMu6MNCBmNSZKmaq4GWsNRpVCCu", environment="us-east4-gcp")

# Create the index
pinecone.create_index(
    name="rag-bot-index",
    dimension=1536,
    metric="cosine"  # Metric for similarity search: cosine, dotproduct, or euclidean
)
index_spec = {
    "replicas": 1,
    "pod_type": "p1"
}

# Check if index exists, create if not
if "rag-bot-index" not in pinecone_client.list_indexes():
    # Define index configuration using PodSpec, including 'environment'
    index_spec = PodSpec(
        pod_type="p1",  # Choose an appropriate pod type
        replicas=1,      # Adjust replicas as needed
        # **Use the same environment here as in the Pinecone() initialization**
        environment="us-east4-gcp"  # Specify your environment
    )
    # Pass 'dimension' as a separate argument and 'spec' for pod configuration
    pinecone_client.create_index("rag-bot-index", dimension=1536, spec=index_spec)

# Get the index
index = pinecone_client.Index("rag-bot-index")

# Set OpenAI API key
openai.api_key = "pcsk_5n25vn_9mtw4PWydANfPXtrWz6uVAa6CQM6zJrHb61DnMu6MNCBmNSZKmaq4GWsNRpVCCu"

def embed_text(text):
    # Initialize the OpenAI client
    client = openai.OpenAI()
    # Use client.embeddings.create for generating embeddings
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    # Return the embedding
    return response.data[0].embedding

# Example knowledge base data
knowledge_base = [
    {"id": "1", "text": "Our business offers 24/7 customer support.", "metadata": {"category": "support"}},
    {"id": "2", "text": "The capital of France is Paris.", "metadata": {"category": "general"}}
]

# Upload embeddings to Pinecone
for item in knowledge_base:
    embedding = embed_text(item["text"])
    index.upsert([(item["id"], embedding, item["metadata"])])

    def embed_text(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response['data'][0]['embedding']

# Example knowledge base data
knowledge_base = [
    {"id": "1", "text": "Our business offers 24/7 customer support.", "metadata": {"category": "support"}},
    {"id": "2", "text": "The capital of France is Paris.", "metadata": {"category": "general"}}
]

# Upload embeddings to Pinecone
for item in knowledge_base:
    embedding = embed_text(item["text"])
    index.upsert([(item["id"], embedding, item["metadata"])])

def retrieve_context(query, top_k=1):
    query_embedding = embed_text(query)
    results = index.query(query_embedding, top_k=top_k, include_metadata=True)
    return " ".join([match["metadata"]["text"] for match in results["matches"]])

def generate_response(query, context):
    prompt = f"Answer the question using the provided context.\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

def qa_pipeline(query):
    context = retrieve_context(query)
    return generate_response(query, context)
